In [1]:
!conda install -c anaconda pandas --yes
!conda install -c anaconda seaborn --yes
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as sp
from scipy.stats import norm,skew,uniform
from scipy import stats
import datetime as dt
%matplotlib inline

Solving environment: done

## Package Plan ##

  environment location: /Users/asamitakeuchi/anaconda3

  added / updated specs:
    - pandas


The following packages will be UPDATED:

  ca-certificates    conda-forge/label/gcc7::ca-certificat~ --> anaconda::ca-certificates-2022.4.26-hecd8cb5_0 

The following packages will be SUPERSEDED by a higher-priority channel:

  certifi            pkgs/main::certifi-2023.7.22-py310hec~ --> anaconda::certifi-2022.6.15-py310hecd8cb5_0 




Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Solving environment: done

# All requested packages already installed.



In [2]:
!conda install -c anaconda nltk --yes
import nltk
nltk.download ()
from nltk.tokenize import sent_tokenize, word_tokenize
import re
from nltk.corpus import stopwords

!conda install -c conda-forge textblob --yes
from textblob import TextBlob
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.manifold import TSNE

from sklearn.compose import ColumnTransformer
!conda install -c "conda-forge/label/gcc7" wordcloud --yes
from wordcloud import WordCloud

import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

import warnings
warnings.filterwarnings('ignore')

Solving environment: done

# All requested packages already installed.

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml
Solving environment: done

## Package Plan ##

  environment location: /Users/asamitakeuchi/anaconda3

  added / updated specs:
    - textblob


The following packages will be UPDATED:

  ca-certificates    anaconda::ca-certificates-2022.4.26-h~ --> conda-forge::ca-certificates-2023.7.22-h8857fd0_0 
  certifi            anaconda/osx-64::certifi-2022.6.15-py~ --> conda-forge/noarch::certifi-2023.7.22-pyhd8ed1ab_0 

The following packages will be SUPERSEDED by a higher-priority channel:

  openssl              pkgs/main::openssl-1.1.1v-hca72f7f_0 --> conda-forge::openssl-1.1.1v-h8a1eda9_0 




Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Solving environment: done

## Package Plan ##

  environment location: /Users/asamitakeuchi/anaconda3

  added / updated specs:
    - wordcloud


The following pack

In [3]:
train = pd.read_csv('/Users/asamitakeuchi/mercari/train2.csv')
train_description = train['item_description']
train_name = train['name']

In [4]:
train_description[:20]

0                                    No description yet
1     This keyboard is in great condition and works ...
2     Adorable top with a hint of lace and a key hol...
3     New with tags. Leather horses. Retail for [rm]...
4             Complete with certificate of authenticity
5     Banana republic bottoms, Candies skirt with ma...
6     Size small but straps slightly shortened to fi...
7     You get three pairs of Sophie cheer shorts siz...
8      Girls Size small Plus green. Three shorts total.
9     I realized his pants are on backwards after th...
10         0.25 oz Full size is 1oz for [rm] in Sephora
11    (5) new vs pink body mists (2.5 oz each) Fresh...
12                                  Xl, great condition
13                                   No description yet
14    Authentic. Suede fringe boots. Great condition...
15    Brand new. Deluxe travel size products. Contai...
16    2 glitter eyeshadows; one in Brass and one in ...
17    Brand new in box Size: Medium Color: Coral

In [5]:
#check the ranking by the number of words of item_description
desc_nums = train_description.apply(lambda x : len(x) if type(x) != float else x )
desc_nums.value_counts()[:20]

18.0    89717
31.0     9587
34.0     9534
37.0     9505
38.0     9374
43.0     9364
36.0     9361
42.0     9328
39.0     9308
35.0     9280
41.0     9250
46.0     9236
30.0     9234
33.0     9208
32.0     9202
44.0     9188
27.0     9188
28.0     9128
49.0     9123
40.0     9113
45.0     9098
48.0     9084
29.0     9082
47.0     9028
19.0     8996
50.0     8967
26.0     8961
20.0     8846
Name: item_description, dtype: int64

In [6]:
#Check the frequently available words


In [7]:
print(type(train_description))

<class 'pandas.core.series.Series'>


## NLP

In [8]:
#Preprocessing
import string
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

stop_words = set(stopwords.words('english'))

# Assuming you have a DataFrame named 'train' with a column 'item_description'
# Replace this with your actual DataFrame
train_description = train['item_description']

def preprocess_text(text):
    if isinstance(text, str):  # Check if the value is a string
        # Remove emojis and unicode text
        text = text.encode('ascii', 'ignore').decode('utf-8')
        
        # Convert to lowercase
        text = text.lower()
        
        # Remove special characters and punctuation
        text = re.sub(r'[^\w\s]', ' ', text)
        
        # Remove numbers
        text = re.sub(r'\d+', '', text)
        
        # Tokenize the text
        tokens = word_tokenize(text)
        
        # Remove stopwords
        stop_words = set(stopwords.words('english'))
        tokens = [word for word in tokens if word not in stop_words]
        
        # Lemmatize the tokens
        lemmatizer = WordNetLemmatizer()
        tokens = [lemmatizer.lemmatize(token) for token in tokens]
        
        # Join tokens back into a cleaned text
        cleaned_text = ' '.join(tokens)
        
        return cleaned_text
    else:
        return ""  # Return an empty string for non-string values

df = pd.DataFrame(train_description)
df['cleaned_description'] = df['item_description'].apply(preprocess_text)

# Display the cleaned descriptions
print(df['cleaned_description'])


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/asamitakeuchi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/asamitakeuchi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/asamitakeuchi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


0                                            description yet
1          keyboard great condition work like came box po...
2          adorable top hint lace key hole back pale pink...
3          new tag leather horse retail rm stand foot hig...
4                          complete certificate authenticity
                                 ...                        
1482530    lace say size small fit medium perfectly never...
1482531        little mermaid handmade dress never worn size
1482532                         used twice still great shape
1482533    one see red orange big red orange one world ma...
1482534         new tag red sparkle firm price free shipping
Name: cleaned_description, Length: 1482535, dtype: object


In [9]:
df['cleaned_description'].dtype

dtype('O')

In [10]:
#Vectorizing
tfidf_vectorizer = TfidfVectorizer(min_df=10,
                             max_features=180000,
                             tokenizer=preprocess_text,
                             ngram_range=(1, 2))
description_matrix = tfidf_vectorizer.fit_transform(list(df['cleaned_description']))
print('Headline after vectorization: \n{}'.format(description_matrix[0]))

Headline after vectorization: 
  (0, 193)	0.19365162707570652
  (0, 744)	0.294861803679301
  (0, 26)	0.3235255570711431
  (0, 428)	0.149659225841063
  (0, 472)	0.1561894035179531
  (0, 303)	0.19483886516520854
  (0, 607)	0.1598610946681362
  (0, 507)	0.3276462527403772
  (0, 304)	0.20853654049901876
  (0, 549)	0.18380211916018072
  (0, 133)	0.25639635998403376
  (0, 572)	0.28111486633666966
  (0, 192)	0.20837628904837072
  (0, 149)	0.19728078259836337
  (0, 737)	0.1500251383527864
  (0, 0)	0.09918147265238658
  (0, 427)	0.1030670957171738
  (0, 456)	0.1078358581679075
  (0, 596)	0.21405690320850407
  (0, 485)	0.1259514963695362
  (0, 286)	0.2101243721475753
  (0, 538)	0.10516304995223984
  (0, 113)	0.11517782831884812
  (0, 567)	0.1086868549202091
  (0, 171)	0.19852588560721193
  (0, 142)	0.11342546931806118


In [ ]:
#LDA
lda_model = LatentDirichletAllocation(n_components=10,
                                      learning_method='online',
                                      max_iter=20,
                                      random_state=42)
X_topics = lda_model.fit_transform(description_matrix)

In [ ]:
n_top_words = 10
topic_summaries = []

topic_word = lda_model.components_  # get the topic words
vocab = tfidf_vectorizer .get_feature_names()

for i, topic_dist in enumerate(topic_word):
    topic_words = np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words+1):-1]
    topic_summaries.append(' '.join(topic_words))
    print('Topic {}: {}'.format(i, ' | '.join(topic_words)))

In [ ]:
description_matrix.sort_values(by=['tfidf'], ascending=True).head(10)